## Двухфакторный дисперсионный анализ. Факторный анализ. Логистическая регрессия

#### Задача 1

Объясняемая переменная y зависит от двух категориальных факторов A и B, причём фактор A имеет 3 уровня, а фактор B - 4 уровня. Результаты наблюдений приведены в таблице:

y = [[2.68, 3.29, 2.88, 4.45],
     [4.12, 4.96, 5.09, 5.22],
     [5.52, 4.50, 5.42, 5.29]]
     
С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную y.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

Для вычисления значений статистик нам вновь понадобятся оценки дисперсий. Они вычисляются похожим образом. Суммы квадратов отклонений:
$$SS_a = k \cdot \displaystyle\sum_{i=1}^m \left( \overline{Y_{i \ast}} - \overline{Y} \right)^2, \:\:
SS_b = m \cdot \displaystyle\sum_{j=1}^k \left( \overline{Y_{\ast j}} - \overline{Y} \right)^2, \:\:
SS_w = \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k \left( y_{ij} - \overline{Y_{i \ast}} - \overline{Y_{\ast j}} + \overline{Y} \right)^2$$

In [32]:
y = np.array([[2.68, 3.29, 2.88, 4.45],
              [4.12, 4.96, 5.09, 5.22],
              [5.52, 4.50, 5.42, 5.29]])

In [33]:
Y_mean = y.mean()
print(f'Среднее значение: {Y_mean}')

Среднее значение: 4.451666666666666


In [34]:
ya1_mean = y[0].mean()
ya2_mean = y[1].mean()
ya3_mean = y[2].mean()

print(f'Среднее значение фактора А: {ya1_mean}, {ya2_mean}, {ya3_mean}')

Среднее значение фактора А: 3.325, 4.8475, 5.1825


In [35]:
yb1_mean = y[:, [0]].mean()
yb2_mean = y[:, [1]].mean()
yb3_mean = y[:, [2]].mean()
yb4_mean = y[:, [3]].mean()

print(f'Среднее значение фактора B: {yb1_mean}, {yb2_mean}, {yb3_mean}, {yb4_mean}')

Среднее значение фактора B: 4.1066666666666665, 4.25, 4.463333333333334, 4.986666666666667


In [37]:
m, k = y.shape
n = y.size
print(f'm = {m}, k = {k}, n = {n}')

m = 3, k = 4, n = 12


In [38]:
SS_a = k * ((ya1_mean - Y_mean)**2 + (ya2_mean - Y_mean)**2 + (ya3_mean - Y_mean)**2)

print(f'SS_a = {SS_a}')

SS_a = 7.8407166666666654


In [39]:
SS_b = m * ((yb1_mean - Y_mean)**2 + (yb2_mean - Y_mean)**2 + (yb3_mean - Y_mean)**2 + (yb4_mean - Y_mean)**2)

print(f'SS_b = {SS_b}')

SS_b = 1.338166666666669


In [40]:
SS_w = ((y[0, [0]] - ya1_mean - yb1_mean + Y_mean)**2) + ((y[1, [0]] - ya2_mean - yb1_mean + Y_mean)**2) + ((y[2, [0]] - ya3_mean - yb1_mean + Y_mean)**2) + ((y[0, [1]] - ya1_mean - yb2_mean + Y_mean)**2) + ((y[1, [1]] - ya2_mean - yb2_mean + Y_mean)**2) + ((y[2, [1]] - ya3_mean - yb2_mean + Y_mean)**2) + ((y[0, [2]] - ya1_mean - yb3_mean + Y_mean)**2) + ((y[1, [2]] - ya2_mean - yb3_mean + Y_mean)**2) + ((y[2, [2]] - ya3_mean - yb3_mean + Y_mean)**2) + ((y[0, [3]] - ya1_mean - yb4_mean + Y_mean)**2) + ((y[1, [3]] - ya2_mean - yb4_mean + Y_mean)**2) + ((y[2, [3]] - ya3_mean - yb4_mean + Y_mean)**2)

print(f'SS_w = {SS_w}')

SS_w = [1.92988333]


Оценки дисперсий:
$$\sigma_a^2 = \dfrac{SS_a}{m - 1}, \:\: \sigma_b^2 = \dfrac{SS_b}{k - 1}, \:\:
\sigma_w^2 = \dfrac{SS_w}{(k - 1) (m - 1)}$$

In [41]:
sigma2_a = SS_a / (m - 1)

print(f'sigma2_a = {sigma2_a}')

sigma2_a = 3.9203583333333327


In [42]:
sigma2_b = SS_b / (k - 1)

print(f'sigma2_b = {sigma2_b}')

sigma2_b = 0.4460555555555563


In [43]:
sigma2_w = SS_w / ((k - 1) * (m - 1))

print(f'sigma2_w = {sigma2_w}')

sigma2_w = [0.32164722]


Напомним, что в двухфакторном дисперсионном анализе мы проверяем две гипотезы (отдельная гипотеза о влиянии каждого из факторов). Итак, статистика для гипотезы о влиянии фактора $a$:
$$F_a = \dfrac{\sigma_a^2}{\sigma_w^2}$$

В предположении верности гипотезы $H_{0a}$ эта статистика имеет распределение Фишера с параметрами $k_{1a} = m - 1$, $k_{2a} = n - m$, где $n = m \cdot k$ — общее число элементов в выборке. Далее, как обычно, строится критическая область (правосторонняя, поскольку распределение Фишера имеет один хвост), и проводится тест.

Аналогично, для гипотезы о влиянии фактора $b$ статистика:
$$F_b = \dfrac{\sigma_b^2}{\sigma_w^2}$$

Она также имеет распределение Фишера, теперь с параметрами $k_{1b} = k - 1$, $k_{2b} = n - k$.

In [44]:
F_a = sigma2_a / sigma2_w
F_b = sigma2_b / sigma2_w
print(f'F_a = {F_a}, \nF_b = {F_b}')

F_a = [12.18837926], 
F_b = [1.38678504]


In [47]:
alpha = 0.05

k1a = m - 1
k2a = m * k - m
t_a = stats.f.ppf(1 - alpha, k1a, k2a)
print(f't_a = {t_a}')

t_a = 4.25649472909375


F_a > t_a - Значение статистики не попало в критическую область, значит влияние фактора A на объясняемую переменную является статистически значимым

In [48]:
k1b = k - 1
k2b = m * k - k
k1b, k2b
t_b = stats.f.ppf(1 - alpha, k1b, k2b)
print(f't_b = {t_b}')

t_b = 4.06618055135116


F_b < t_b - Значение статистики попало в критическую область, значит влияние фактора B на объясняемую переменную не очень значимо

#### Задача 2

Прикрепите ссылку на курсовой проект на github (см. выбор темы в ДЗ к Уроку 6).

In [ ]:
https://github.com/ViktorOtroschenko/Probability-theory/branches